In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from nurisc import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from nurisc.matching import matching, matching_dataset
from nurisc.models import Config2D, nurisc2D, nuriscData2D

np.random.seed(42)
lbl_cmap = random_label_cmap()

C:\Users\es255022\AppData\Local\Temp\ipykernel_13580\3250185321.py:5: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  matplotlib.rcParams["image.interpolation"] = None


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.
Model: "denseUnet_fNLA"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 528, 240, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv00_block1_conv (Conv2D)    (None, 528, 240, 5)  45          ['input_1[0][0]']                
                                                                                                  
 conv00_block1_bn (BatchNormali  (None, 528, 240, 5)  35         ['conv00_block1_conv[0][0]']     
 zation)                                                                          

nadam.py (78): The `lr` argument is deprecated, use `learning_rate` instead.


 conv10_block8_2_bn (BatchNorma  (None, 264, 120, 5)  35         ['conv10_block8_2_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv10_block8_2_actv (Activati  (None, 264, 120, 5)  0          ['conv10_block8_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv10_block8_drop (Dropout)   (None, 264, 120, 5)  0           ['conv10_block8_2_actv[0][0]']   
                                                                                                  
 conv10_block8_concat (Concaten  (None, 264, 120, 50  0          ['conv10_block7_concat[0][0]',   
 ate)                           )                                 'conv10_block8_drop[0][0]']     
          

In [2]:
#CoNSeP
from cv2 import imread
X= sorted(glob('C:/Users/es255022/Downloads/consep_dataset/CoNSeP/Train/Images/*.png'))
print(len(X))
X = list(map(imread,X))
from scipy.io import  loadmat
Y1= sorted(glob('C:/Users/es255022/Downloads/consep_dataset/CoNSeP/Train/Labels/*.mat'))
print(len(Y1))
# np.int(Y1)
Y1=list(map(loadmat,Y1))

# Y = [y[...,0] for y in Y]
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]

27
27


In [4]:
Y=[]
Z=[]
for i in range(len(Y1)):
 for k, v in Y1[i].items():
    if k=='inst_map':
      
        Y.append(v)
        print(v)
    if k=='type_map':
       Z.append(v)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ... 995. 995. 995.]
 [  0.   0.   0. ... 995. 995. 995.]
 [  0.   0.   0. ... 995. 995. 995.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[  0.   0.   0. ... 453. 453. 453.]
 [  0.   0.   0. ... 453. 453. 453.]
 [  0.   0.   0. ... 453. 453. 453.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 

In [5]:
Y=np.asarray(Y)

In [6]:
Y=Y.astype('int32')

In [7]:
Z=np.asarray(Z)
Z=Z.astype('int32')

In [8]:
Z[Z == 4] = 3
Z[Z == 5] = 4
Z[Z == 6] = 4
Z[Z == 7] = 4

In [9]:
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()
X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X)]
Y = [fill_label_holes(y) for y in tqdm(Y)]

Normalizing image channels independently.


100%|██████████| 27/27 [00:01<00:00, 19.20it/s]


In [10]:
ind_train=27
X_trn, Y_trn = [X[i] for i in range(27)], [Y[i] for i in range(27)] 

In [11]:
#CoNSeP val
from cv2 import imread
X= sorted(glob('C:/Users/es255022/Downloads/consep_dataset/CoNSeP/Test/Images/*.png'))
print(len(X))
X_val = list(map(imread,X))
from scipy.io import  loadmat
Y1= sorted(glob('C:/Users/es255022/Downloads/consep_dataset/CoNSeP/Test/Labels/*.mat'))
print(len(Y1))
Y1=list(map(loadmat,Y1))
# Y = [y[...,0] for y in Y]
n_channel = 1 if X_val[0].ndim == 2 else X_val[0].shape[-1]

14
14


In [12]:
Y_val=[]
Z_val=[]
for i in range(len(Y1)):
 for k, v in Y1[i].items():
    if k=='inst_map':
        Y_val.append(v)
    if k=='type_map':
        Z_val.append(v)
      

In [13]:
Y_val=np.asarray(Y_val)
Y_val=Y_val.astype('int32')
Z_val=np.asarray(Z_val)
Z_val=Z_val.astype('int32')

In [14]:
Z_val[Z_val == 4] = 3
Z_val[Z_val == 5] = 4
Z_val[Z_val == 6] = 4
Z_val[Z_val == 7] = 4

In [15]:
# assert len(X) > 1, "not enough training data"
# rng = np.random.RandomState(42)
# ind = rng.permutation(len(X))
# n_val = max(1, int(round(0.3 * len(ind))))
# ind_train, ind_val = ind[:-n_val], ind[-n_val:]
# X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]

print('number of images: %3d' % len(X_trn))
print('- training:       %3d' % len(Y_trn))
# print('- validation:     %3d' % len(X_val))

number of images:  27
- training:        27


In [16]:
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()
X_val = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X_val)]
Y_val = [fill_label_holes(y) for y in tqdm(Y_val)]

Normalizing image channels independently.


100%|██████████| 14/14 [00:00<00:00, 18.74it/s]


In [17]:
cls_trn=[]
for i in range(27):
    cls_trn.append(dict(zip(Y[i].flatten(), Z[i].flatten())))
cls_val=[]
for i in range(14):
    cls_val.append(dict(zip(Y_val[i].flatten(), Z_val[i].flatten())))    

In [18]:
from nurisc.plot import render_label
def plot_img_label(img, lbl, img_title="image", lbl_title="label", **kwargs):
  

    fig, (ai,al) = plt.subplots(1,2, figsize=(17,7), gridspec_kw=dict(width_ratios=(1.,1)))
    im = ai.imshow(img, cmap='gray')
    #fig.colorbar(im, ax = ai)
    ai.set_title(img_title)
    al.imshow(render_label(lbl, .8*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8,cmap=lbl_cmap))
    al.set_title(lbl_title)
    
    plt.tight_layout()
    for a in ai,al:
        a.axis("off")
    return ai,al

In [19]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
import json
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from nurisc import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from nurisc.matching import matching, matching_dataset
from nurisc.models import Config2D, nurisc2D, nuriscData2D
from nurisc.utils import mask_to_categorical
from nurisc.plot import render_label



np.random.seed(0)
lbl_cmap = matplotlib.cm.nipy_spectral_r
lbl_cmap_classes =  matplotlib.cm.Accent_r

def plot_img_label(img, lbl, cls_dict, n_classes=4, img_title="image", lbl_title="label", cls_title="classes", **kwargs):
    c = mask_to_categorical(lbl, n_classes=n_classes, classes=cls_dict)
    res = np.zeros(lbl.shape, np.uint16)
    for i in range(1,c.shape[-1]):
        m = c[...,i]>0
        res[m] = i
    class_img = lbl_cmap_classes(res)
    # class_img[...,:3][res==0] = 0
    # class_img[...,-1][res==0] = 1

    fig, (ai,al,ac) = plt.subplots(1,3, figsize=(17,7), gridspec_kw=dict(width_ratios=(1.,1,1)))
    im = ai.imshow(img, cmap='gray')
    #fig.colorbar(im, ax = ai)
    ai.set_title(img_title)
    al.imshow(render_label(lbl, .5*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8,cmap=lbl_cmap))
    al.set_title(lbl_title)
    ac.imshow(class_img)
    ac.imshow(render_label(res, .5*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8, cmap=lbl_cmap_classes))
    ac.set_title(cls_title)
    plt.tight_layout()
    for a in ai,al,ac:
        a.axis("off")
    return ai,al,ac

3258213507.py (5): Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.


In [ ]:
ax = plot_img_label(X_trn[1],Y_trn[1], cls_trn[1])

In [20]:
np.random.seed(0)
lbl_cmap = matplotlib.cm.tab10
lbl_cmap_classes = matplotlib.cm.tab10

def plot_img_label(img, lbl, cls_dict, n_classes=4, img_title="image", lbl_title="label", cls_title="classes", **kwargs):
    c = mask_to_categorical(lbl, n_classes=n_classes, classes=cls_dict)
    res = np.zeros(lbl.shape, np.uint16)
    for i in range(1,c.shape[-1]):
        m = c[...,i]>0
        res[m] = i
    class_img = lbl_cmap_classes(res)
    # class_img[...,:3][res==0] = 0
    # class_img[...,-1][res==0] = 1

    fig, (ai,al,ac) = plt.subplots(1,3, figsize=(17,7), gridspec_kw=dict(width_ratios=(1.,1,1)))
    im = ai.imshow(img, cmap='gray')
    #fig.colorbar(im, ax = ai)
    ai.set_title(img_title)
    al.imshow(render_label(lbl, .3*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8,cmap=lbl_cmap))
    al.set_title(lbl_title)
    ac.imshow(class_img)
    ac.imshow(render_label(res, .3*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8, cmap=lbl_cmap_classes))
    ac.set_title(cls_title)
    plt.tight_layout()
    for a in ai,al,ac:
        a.axis("off")
    return ai,al,ac

In [ ]:
ax = plot_img_label(X_trn[1],Y_trn[1], cls_trn[1])

In [ ]:
ax = plot_img_label(X_trn[1],Y_trn[1], cls_trn[1])

In [23]:
print(Config2D.__doc__)

Configuration for a :class:`nurisc2D` model.

    Parameters
    ----------
    axes : str or None
        Axes of the input images.
    n_rays : int
        Number of radial directions for the nuclei.
    n_channel_in : int
        Number of channels of given input image (default: 1).

    grid : (int,int)
        Subsampling factors (must be powers of 2) for each of the axes.
        Model will predict on a subsampled grid for increased efficiency and larger field of view.
    n_classes : None or int
        Number of object classes to use for multi-class prediction (use None to disable)
    backbone : str
        Name of the neural network architecture to be used as backbone.
    kwargs : dict
        Overwrite (or add) configuration attributes (see below).


    Attributes
    ----------
    unet_n_depth : int
        Number of U-Net resolution levels (down/up-sampling layers).
    unet_kernel_size : (int,int)
        Convolution kernel size for all (U-Net) convolution layers.
    

In [21]:
n_rays = 32
n_classes=4
# Use OpenCL-based computations for data generator during training (requires 'gputools')
use_gpu = True and gputools_available()

# Predict on subsampled grid for increased efficiency and larger field of view
grid = (2,2)

conf = Config2D (
    n_rays       = n_rays,
    grid         = grid,
    use_gpu      = True,
    n_channel_in = n_channel,
    n_classes    = n_classes,
   # train_class_weights=(1,2.3581024244032336,3.8217535910612446,3.078248267944116,7.188182340917902,3.485527854099882),
   #  train_class_loss='cce_dice',
    train_batch_size=4,
    backbone='ffnet',
    # train_learning_rate=0.0001
    train_patch_size=(512,512)
    # set the number of object classes
)
print(conf)
vars(conf)

If you want to compute separable approximations, please install it with
pip install scikit-tensor-py3
Config2D(n_dim=2, axes='YXC', n_channel_in=3, n_channel_out=33, train_checkpoint='weights_best.h5', train_checkpoint_last='weights_last.h5', train_checkpoint_epoch='weights_now.h5', n_rays=32, grid=(2, 2), backbone='ffnet', n_classes=4, unet_n_depth=4, unet_kernel_size=(3, 3), unet_n_filter_base=32, unet_n_conv_per_depth=2, unet_pool=(2, 2), unet_activation='relu', unet_last_activation='relu', unet_batch_norm=True, unet_dropout=0.0, unet_prefix='', net_conv_after_unet=128, head_blocks=2, net_input_shape=(None, None, 3), net_mask_shape=(None, None, 1), train_shape_completion=False, train_completion_crop=32, train_patch_size=(512, 512), train_background_reg=0.0001, train_foreground_only=0.9, train_sample_cache=True, train_dist_loss='mae', train_class_loss='cce_tversky', train_loss_weights=(1, 0.2, 1), train_class_weights=(1, 1, 1, 1, 1), train_epochs=400, train_steps_per_epoch=100, train

{'n_dim': 2,
 'axes': 'YXC',
 'n_channel_in': 3,
 'n_channel_out': 33,
 'train_checkpoint': 'weights_best.h5',
 'train_checkpoint_last': 'weights_last.h5',
 'train_checkpoint_epoch': 'weights_now.h5',
 'n_rays': 32,
 'grid': (2, 2),
 'backbone': 'ffnet',
 'n_classes': 4,
 'unet_n_depth': 4,
 'unet_kernel_size': (3, 3),
 'unet_n_filter_base': 32,
 'unet_n_conv_per_depth': 2,
 'unet_pool': (2, 2),
 'unet_activation': 'relu',
 'unet_last_activation': 'relu',
 'unet_batch_norm': True,
 'unet_dropout': 0.0,
 'unet_prefix': '',
 'net_conv_after_unet': 128,
 'head_blocks': 2,
 'net_input_shape': (None, None, 3),
 'net_mask_shape': (None, None, 1),
 'train_shape_completion': False,
 'train_completion_crop': 32,
 'train_patch_size': (512, 512),
 'train_background_reg': 0.0001,
 'train_foreground_only': 0.9,
 'train_sample_cache': True,
 'train_dist_loss': 'mae',
 'train_class_loss': 'cce_tversky',
 'train_loss_weights': (1, 0.2, 1),
 'train_class_weights': (1, 1, 1, 1, 1),
 'train_epochs': 400,

In [22]:
model = nurisc2D(config=None, name='consep_ffnet_256', basedir='C:/Users/es255022/Downloads')

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.612443, nms_thresh=0.3.


In [23]:
model1 = nurisc2D(config=None, name='consep_unet', basedir='C:/Users/es255022/Downloads/')

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.563833, nms_thresh=0.3.


In [24]:
median_size = calculate_extents(list(Y), np.median)
fov = np.array(model._axes_tile_overlap('YX'))
print(f"median object size:      {median_size}")
print(f"network field of view :  {fov}")
print('done')
if any(median_size > fov):
    print("WARNING: median object size larger than field of view of the neural network.")

structured_function.py (264): Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.


1/1 [==============================] - 1s 542ms/step
median object size:      [18. 18.]
network field of view :  [128 128]
done


In [25]:
def random_fliprot(img, mask): 
    assert img.ndim >= mask.ndim
    axes = tuple(range(mask.ndim))
    perm = tuple(np.random.permutation(axes))
    img = img.transpose(perm + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(perm) 
    for ax in axes: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img
# def random_crop(img,mask):
#     assert img.ndim >= mask.ndim
#     axes = tuple(range(mask.ndim))
#     perm = tuple(np.random.permutation(axes))
#     img= iaa.Crop(percent=(0, 0.3)) # crop image
#     mask= iaa.Crop(percent=(0, 0.3))
#     return img, mask 
# def random_img_shearing(img,mask):
#   shear = iaa.Affine(shear=(0,40))
#   img = shear.augment_image(img)
#   mask = shear.augment_image(mask)
#   return img, mask
def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    x, y = random_fliprot(x, y)
    # x,y =random_crop(x,y)
    # x,y =random_img_shearing(x,y)
    x = random_intensity_change(x)
    # add some gaussian noise
    sig = 0.02*np.random.uniform(0,1)
    x = x + sig*np.random.normal(0,1,x.shape)
    return x, y

In [30]:
quick_demo = False

if quick_demo:
    print (
        "NOTE: This is only for a quick demonstration!\n"
        "      Please set the variable 'quick_demo = False' for proper (long) training.",
        file=sys.stderr, flush=True
    )
    model.train(X_trn, Y_trn, validation_data=(X_val,Y_val), augmenter=augmenter,
                epochs=1, steps_per_epoch=10)

    print("====> Stopping training and loading previously trained demo model from disk.", file=sys.stderr, flush=True)
    model = nurisc2D.from_pretrained('2D_cryonuseg')
else:
    model.train(X_trn, Y_trn, classes=cls_trn, validation_data=(X_val,Y_val,cls_val), augmenter=augmenter,epochs=100, steps_per_epoch=100)
    print('done')
None;

1.0 0.7
class_loss:    cce_tversky
class_weights: [1, 1, 1, 1, 1]
focal_gamma:   0
Epoch 1/100


KeyboardInterrupt: 

In [ ]:
quick_demo = False
if quick_demo:
    model.optimize_thresholds(X_val[:2], Y_val[:2])
else:
    model.optimize_thresholds(X_val, Y_val)

1/1 [==============================] - 0s 22ms/step


NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:42<00:14,  2.81s/it, 0.606 -> 0.451]


Using optimized values: prob_thresh=0.576781, nms_thresh=0.3.
Saving to 'thresholds.json'.


In [ ]:
Y_val_pred1, res_val_pred1 = tuple(zip(*[model1.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)
              for x in tqdm(X_val[:])]))

In [ ]:
Y_val_pred, res_val_pred = tuple(zip(*[model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)
              for x in tqdm(X_val[:])]))


In [86]:
 def plot_examples(colormaps):
    """
    Helper function to plot data with associated colormap.
    """
    np.random.seed(19680801)
    data = np.random.randn(30, 30)
    n = len(colormaps)
    fig, axs = plt.subplots(1, n, figsize=(n * 2 + 2, 3),
                            constrained_layout=True, squeeze=False)
    for [ax, cmap] in zip(axs.flat, colormaps):
        psm = ax.pcolormesh(data, cmap=cmap, rasterized=True, vmin=-4, vmax=4)
        fig.colorbar(psm, ax=ax)
    plt.show()

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
cmap = ListedColormap(["darkorange","yellow" , "green", "blue","red"])
plot_examples([cmap])

In [44]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
cmap = ListedColormap([ "lemonchiffon"])

In [35]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
import json
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from nurisc import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from nurisc.matching import matching, matching_dataset
from nurisc.models import Config2D, nurisc2D, nuriscData2D
from nurisc.utils import mask_to_categorical
from nurisc.plot import render_label



np.random.seed(0)
lbl_cmap = matplotlib.cm.gist_rainbow
lbl_cmap_classes = cmap

def plot_img_label(img, lbl, cls_dict, n_classes=4, img_title="image", lbl_title="label", cls_title="classes", **kwargs):
    c = mask_to_categorical(lbl, n_classes=n_classes, classes=cls_dict)
    res = np.zeros(lbl.shape, np.uint16)
    for i in range(1,c.shape[-1]):
        m = c[...,i]>0
        res[m] = i
    class_img = lbl_cmap_classes(res)
    # class_img[...,:3][res==0] = 0
    # class_img[...,-1][res==0] = 1

    fig, (ai,al,ac) = plt.subplots(1,3, figsize=(17,7), gridspec_kw=dict(width_ratios=(1.,1,1)))
    im = ai.imshow(img, cmap='gray')
    #fig.colorbar(im, ax = ai)
    ai.set_title(img_title)
    al.imshow(render_label(lbl, .7*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8,cmap=lbl_cmap))
    al.set_title(lbl_title)
    ac.imshow(class_img)
    ac.imshow(render_label(res, .7*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8, cmap=lbl_cmap_classes))
    ac.set_title(cls_title)
    plt.tight_layout()
    for a in ai,al,ac:
        a.axis("off")
    return ai,al,ac

2203971208.py (5): Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.


In [33]:
cmap2 = matplotlib.colors.ListedColormap ( np.random.rand ( 256,3))

In [57]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
import json
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from nurisc import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from nurisc.matching import matching, matching_dataset
from nurisc.models import Config2D, nurisc2D, nuriscData2D
from nurisc.utils import mask_to_categorical
from nurisc.plot import render_label



np.random.seed(0)
lbl_cmap =ListedColormap([ "lime"])
lbl_cmap_classes = cmap

def plot_img_label1(img, lbl, cls_dict, n_classes=4, img_title="image", lbl_title="label", cls_title="classes", **kwargs):
    c = mask_to_categorical(lbl, n_classes=n_classes, classes=cls_dict)
    res = np.zeros(lbl.shape, np.uint16)
    for i in range(1,c.shape[-1]):
        m = c[...,i]>0
        res[m] = i
    class_img = lbl_cmap_classes(res)
    # class_img[...,:3][res==0] = 0
    # class_img[...,-1][res==0] = 1

    fig, (ai,al,ac) = plt.subplots(1,3, figsize=(17,7), gridspec_kw=dict(width_ratios=(1.,1,1)))
    im = ai.imshow(img, cmap='gray')
    #fig.colorbar(im, ax = ai)
    ai.set_title(img_title)
    al.imshow(render_label(lbl, .99*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8,cmap=lbl_cmap))
    al.set_title(lbl_title)
    ac.imshow(class_img)
    ac.imshow(render_label(res, .99*normalize(img, clip=True), normalize_img=False, alpha_boundary=.8, cmap=lbl_cmap_classes))
    ac.set_title(cls_title)
    plt.tight_layout()
    for a in ai,al,ac:
        a.axis("off")
    return ai,al,ac

4270252184.py (5): Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.


In [28]:
# the class object ids are stored in the 'results' dict and correspond to the label ids in increasing order 
def class_from_res(res):
    cls_dict = dict((i+1,c) for i,c in enumerate(res['class_id']))
    return cls_dict


In [29]:
def test_img_res( lbl, cls_dict, n_classes=4):
    c = mask_to_categorical(lbl, n_classes=n_classes, classes=cls_dict)
    res = np.zeros(lbl.shape, np.uint16)

    for i in range(1,c.shape[-1]):
        m = c[...,i]>0
        res[m] = i

    print(res.shape)
    print(res)
    print(np.unique(res))

    return res

In [ ]:
i3=6

plot_img_label1(X_val[i3], Y_val[i3], cls_val[i3],            lbl_title="GroundTruth" ,cls_title="GroundTruth_Classes");
plot_img_label1(X_val[i3], Y_val_pred1[i3],    class_from_res(res_val_pred1[i3]), lbl_title=" Baseline", cls_title="Baseline_Classes");
plot_img_label1(X_val[i3], Y_val_pred[i3],    class_from_res(res_val_pred[i3]), lbl_title=" Proposed_Predicted", cls_title="Proposed_Classes");

In [ ]:
i3=0

plot_img_label1(X_val[i3], Y_val[i3], cls_val[i3],            lbl_title="GroundTruth" ,cls_title="GroundTruth_Classes");
plot_img_label1(X_val[i3], Y_val_pred1[i3],    class_from_res(res_val_pred1[i3]), lbl_title=" Baseline", cls_title="Baseline_Classes");
plot_img_label1(X_val[i3], Y_val_pred[i3],    class_from_res(res_val_pred[i3]), lbl_title=" Proposed_Predicted", cls_title="Proposed_Classes")

In [ ]:
i3=10

plot_img_label1(X_val[i3], Y_val[i3], cls_val[i3],            lbl_title="GroundTruth" ,cls_title="GroundTruth_Classes");
plot_img_label1(X_val[i3], Y_val_pred1[i3],    class_from_res(res_val_pred1[i3]), lbl_title=" Baseline", cls_title="Baseline_Classes");
plot_img_label1(X_val[i3], Y_val_pred[i3],    class_from_res(res_val_pred[i3]), lbl_title=" Proposed_Predicted", cls_title="Proposed_Classes")

In [ ]:
i3=2
plot_img_label(X_val[i3], Y_val[i3], cls_val[i3],            lbl_title="GroundTruth" ,cls_title="GroundTruth_Classes");
plot_img_label(X_val[i3], Y_val_pred1[i3],    class_from_res(res_val_pred1[i3]), lbl_title=" Baseline", cls_title="Baseline_Classes");
plot_img_label(X_val[i3], Y_val_pred[i3],    class_from_res(res_val_pred[i3]), lbl_title=" Proposed_Predicted", cls_title="Proposed_Classes");

In [ ]:
i3 = 3
n_classes = 7
plot_img_label(X_val[i3], Y_val[i3], cls_val[i3],            lbl_title="GroundTruth" ,cls_title="GroundTruth_Classes");
plot_img_label(X_val[i3], Y_val_pred1[i3],    class_from_res(res_val_pred1[i3]), lbl_title=" Baseline", cls_title="Baseline_Classes");
plot_img_label(X_val[i3], Y_val_pred[i3],    class_from_res(res_val_pred[i3]), lbl_title=" Proposed_Predicted", cls_title="Proposed_Classes");

In [ ]:
i3=6
plot_img_label(X_val[i3], Y_val[i3], cls_val[i3],            lbl_title="GroundTruth" ,cls_title="GroundTruth_Classes");
plot_img_label(X_val[i3], Y_val_pred1[i3],    class_from_res(res_val_pred1[i3]), lbl_title=" Baseline", cls_title="Baseline_Classes");
plot_img_label(X_val[i3], Y_val_pred[i3],    class_from_res(res_val_pred[i3]), lbl_title=" Proposed_Predicted", cls_title="Proposed_Classes");

In [ ]:
i3=5
plot_img_label(X_val[i3], Y_val[i3], cls_val[i3],            lbl_title="GroundTruth" ,cls_title="GroundTruth_Classes");
plot_img_label(X_val[i3], Y_val_pred1[i3],    class_from_res(res_val_pred1[i3]), lbl_title=" Baseline", cls_title="Baseline_Classes");
plot_img_label(X_val[i3], Y_val_pred[i3],    class_from_res(res_val_pred[i3]), lbl_title=" Proposed_Predicted", cls_title="Proposed_Classes");

In [ ]:

i3 = 6
n_classes = 7
label, res = model.predict_instances(X_val[i3], n_tiles=model._guess_n_tiles(X_val[i3]))
plot_img_label(X_val[i3], Y_val[i3], cls_val[i3], lbl_title="GroundTruth", cls_title="GroundTruth_Classes")
plot_img_label(X_val[i3], label, class_from_res(res), lbl_title="Proposed_Predicted", cls_title="Proposed_Classes");

In [163]:

taus1 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
stats1 = [matching_dataset(Y_val, Y_val_pred1, thresh=t, show_progress=False) for t in tqdm(taus1)]

100%|██████████| 9/9 [00:06<00:00,  1.46it/s]


In [164]:
stats1[taus1.index(0.5)]

DatasetMatching(criterion='iou', thresh=0.5, fp=1990, tp=5293, fn=3484, precision=0.726760950157902, recall=0.6030534351145038, accuracy=0.4915946874709761, f1=0.6591531755915317, n_true=8777, n_pred=7283, mean_true_score=0.4407453255568086, mean_matched_score=0.7308561727587586, panoptic_quality=0.4817461671746089, segmentation_quality=0.7308561727587586, detection_quality=0.6591531755915317, by_image=False)

In [165]:
taus = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
stats = [matching_dataset(Y_val, Y_val_pred, thresh=t, show_progress=False) for t in tqdm(taus)]

100%|██████████| 9/9 [00:05<00:00,  1.53it/s]


In [166]:
stats[taus.index(0.5)]

DatasetMatching(criterion='iou', thresh=0.5, fp=1894, tp=5576, fn=3201, precision=0.7464524765729585, recall=0.6352967984504956, accuracy=0.5225377190516353, f1=0.6864036437496153, n_true=8777, n_pred=7470, mean_true_score=0.4828151847610743, mean_matched_score=0.7599836579354284, panoptic_quality=0.5216555519970394, segmentation_quality=0.7599836579354284, detection_quality=0.6864036437496153, by_image=False)

In [ ]:
taus1 = [ 0.3,0.4, 0.5, 0.6, 0.7, 0.8]
stats1 = [matching_dataset(Y_val, Y_val_pred, thresh=t, show_progress=False) for t in tqdm(taus1)]
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(25,7))
colors = "bgr"
i= -1
j=0
for m in (  'f1','accuracy','panoptic_quality'):
    ax1.bar(taus1, [s._asdict()[m] for s in stats1], width=0.05, label=m, color=colors[i])
    i=i+1

ax1.set_xlabel(r'CoNSeP Quantitative Results')
ax1.set_ylabel('Evaluation Metrics')
# t="CPM-17 Quantitative Results"
# ax1.title(t)
# ax1.grid()
ax1.legend()
L=ax1.legend()
L.get_texts()[0].set_text('Dice Similarity Coefficient')
L.get_texts()[1].set_text('Accuracy')
L.get_texts()[2].set_text('Panoptic Quality')

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,5))
colors = "brcymkw"
i= 0
for m in ('precision', 'recall'):
    ax1.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m,c=colors[i],marker='o')
    i=i+1
ax1.set_xlabel(r'CoNSeP Quantitative Results')
ax1.set_ylabel('Evaluation Metrics')
# t="CPM-17 Quantitative Results"
# ax1.title(t)
# ax1.grid()
ax1.legend()
L=ax1.legend()
L.get_texts()[0].set_text('Precision')
L.get_texts()[1].set_text('Recall')

colors1 = "ygrmgrcy"
j=0
for m in ('fp', 'tp', 'fn'):
    ax2.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m,c=colors1[j],marker='^')
    j=j+1
ax2.set_xlabel(r'Optimizied Threshold $\tau$ Nuclei Stats')
ax2.set_ylabel('Number #')
# ax2.grid()
ax2.legend();
L1=ax2.legend()
L1.get_texts()[0].set_text('False Positive')
L1.get_texts()[1].set_text('True Positive')
L1.get_texts()[2].set_text('False Negative')

In [ ]:
from nltk import flatten
cls_val_tmp_dict = [[0 for x in range(66536)] for y in range(len(cls_val))]
cls_val_dict = [[0 for x in range(66536)] for y in range(len(cls_val))]
cls_tmp_val=np.zeros_like(Y_val)
for i in range(len(cls_val)):

   cls_tmp_val[i]=test_img_res( Y_val_pred1[i],    class_from_res(res_val_pred1[i]));
# cls=[cls[...,4]for cls in val_data]
cls_val_new = Z_val
for i in range(len(cls_val)):
    cls_val_dict[i]=cls_val_new[i].flatten()
    cls_val_tmp_dict[i]=cls_tmp_val[i].flatten()
tr=list(cls_val_tmp_dict)
tr1=flatten(tr)
z=np.concatenate( tr1, axis=0 )
tr2=list(cls_val_dict)
tr3=flatten(tr2)
w=np.concatenate( tr3, axis=0 )

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
target_names=['Nolabel', 'Epithelial', 'Inflammatory','Spindle','Misc']
cm = confusion_matrix(w, z)
# Normalise
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.set(font_scale=1.5)
sns.heatmap(cmn, annot=True, fmt='.3f', xticklabels=target_names, yticklabels=target_names, cmap=plt.cm.Blues)
plt.xticks(target_names, fontsize=15)
plt.yticks(target_names,fontsize=15)
plt.ylabel('Actual')
plt.xlabel('Predicted')

plt.show(block=False)

In [173]:
from nltk import flatten
cls_val_tmp_dict = [[0 for x in range(66536)] for y in range(len(cls_val))]
cls_val_dict = [[0 for x in range(66536)] for y in range(len(cls_val))]
cls_tmp_val=np.zeros_like(Y_val)
for i in range(len(cls_val)):

   cls_tmp_val[i]=test_img_res( Y_val_pred1[i],    class_from_res(res_val_pred1[i]));
# cls=[cls[...,4]for cls in val_data]
cls_val_new = Z_val
for i in range(len(cls_val)):
    cls_val_dict[i]=cls_val_new[i].flatten()
    cls_val_tmp_dict[i]=cls_tmp_val[i].flatten()
tr=list(cls_val_tmp_dict)
tr1=flatten(tr)
z=np.concatenate( tr1, axis=0 )
tr2=list(cls_val_dict)
tr3=flatten(tr2)
w=np.concatenate( tr3, axis=0 )

(1000, 1000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 4 0 0]
 [0 0 0 ... 4 4 4]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0 1 2 3 4]
(1000, 1000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 2]
 [0 0 0 ... 2 2 0]]
[0 2 3 4]
(1000, 1000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0 1 2 3 4]
(1000, 1000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0 1 2 3 4]
(1000, 1000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 2 2 0]
 [0 0 0 ... 2 2 0]
 [0 0 0 ... 2 0 0]]
[0 1 2 3 4]
(1000, 1000)
[[0 0 0 ... 0 0 0]
 [0 0 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0 1 2 3 4]
(1000, 1000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
target_names=['Nolabel', 'Epithelial', 'Inflammatory','Spindle','Misc']
cm = confusion_matrix(w, z)
# Normalise
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=target_names, yticklabels=target_names, cmap=plt.cm.Blues)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Baseline')
plt.show(block=False)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,5))
colors = "bgrcymkw"
i= 0
for m in ('precision', 'recall', 'accuracy', 'f1', 'panoptic_quality'):
    ax1.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m,c=colors[i],marker='o')
    i=i+1
ax1.set_xlabel(r'CoNSeP Quantitative Results')
ax1.set_ylabel('Evaluation Metrics')
# t="CPM-17 Quantitative Results"
# ax1.title(t)
# ax1.grid()
ax1.legend()
L=ax1.legend()
L.get_texts()[0].set_text('Precision')
L.get_texts()[1].set_text('Recall')
L.get_texts()[2].set_text('Accuracy')
L.get_texts()[3].set_text('Dice Similarity Coefficient')
L.get_texts()[4].set_text('Panoptic Quality')
for m in ('fp', 'tp', 'fn'):
    ax2.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m)
ax2.set_xlabel(r'IoU threshold $\tau$')
ax2.set_ylabel('Number #')
ax2.grid()
ax2.legend();